In [ ]:
# ドライブのマウント
from google.colab import drive
drive.mount('/content/drive')

# 適宜自分のApplicationPATHまで変更
APP_PATH = '/content/drive/Othercomputers/LAPTOP-3BHDM0TI/windows_research/workspace/research/carbon-steel-classification'


import numpy as np
import matplotlib.pyplot as plt
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import TensorDataset, DataLoader
from torch import nn
import torch.nn.functional as F
from torch import optim

from sklearn.model_selection import train_test_split

import os
import sys
import cv2
from PIL import Image
import pandas as pd


os.chdir(APP_PATH)
!pwd
!ls -al

Mounted at /content/drive
/content/drive/Othercomputers/LAPTOP-3BHDM0TI/windows_research/workspace/research/carbon-steel-classification
total 28
drwx------ 2 root root 4096 May  8 09:43 app
drwx------ 2 root root 4096 May  8 09:52 bin
drwx------ 2 root root 4096 May  9 20:26 data
drwx------ 2 root root 4096 May 10 06:28 dataset_npz
drwx------ 2 root root 4096 May  8 09:16 .git
-rw------- 1 root root   48 May  8 09:58 .gitignore
-rw------- 1 root root  265 May  8 09:51 Makefile
-rw------- 1 root root  358 May  8 09:37 README.md
drwx------ 2 root root 4096 May  8 09:38 tmp
-rw------- 1 root root 2220 May 18 08:23 TODO.txt


In [ ]:
# ファイルの実行(ノートブック側でランタイムの再接続 or reloadをする)
# importで実行するときはファイルのパスを通す必要がある
sys.path.append(APP_PATH + '/bin')

# setupなどのOSに関するコマンド実行はその定義ファイルをコマンドで実行
# 関数などのモジュールはimportで扱えるようにして実行
# !python bin/setup.py

In [ ]:
# 画像データ(npzのバイナリファイル)の読み込み

# 対物レンズ10倍。
# 入力サイズ960px
# x.1の画像のみのデータセット


datasets_npz_x1 = np.load(APP_PATH + '/dataset_npz/x.1_dataset/sc_x10_960_x1.npz')
# datasets_npz_x3 = np.load(APP_PATH + '/dataset_npz/x.1_dataset/sc_x10_960_x1.npz')
# datasets_npz_all = np.load(APP_PATH + '/dataset_npz/x.1_dataset/sc_x10_960_x1.npz')


CATEGORIES=["S10C","S15C","S25C","S35C","S45C","S55C"]


# Xはdata, Yはtargetのこと
# print(datasets_npz_x1)
# print(datasets_npz_x1.files)

# データを3次元のtensor([c h w])に変換(1次元と3次元で比較)
images = datasets_npz_x1['x']
labels = datasets_npz_x1['y']

# transpose_images = np.empty(shape=1, dtype=float)
# print("transpose_images: {}".format(transpose_images))
images_list = []
labels_list = []

NUM_DATA = 10

for i, image in enumerate(images):
    # CPUが使い果たされてしまうので200までを使用
    if i == NUM_DATA:
        break

    print('image-type：', type(image))
    # 標準化もここで行なう？？
    image = Image.fromarray(image)
    print("image dtype: {}".format(type(image)))
    image = image.convert("RGB")
    print("image mode: {}".format(image.mode)) # むりやり3チャンネルにしているので表記上はRGBだが実際はL
    image = np.asarray(image, np.uint8)
    print("image shape: {}".format(image.shape))
    image = np.transpose(image, (2, 0, 1))
    print("image transpose shape: {}".format(image.shape))
    # transpose_images = np.append(transpose_images, image, axis=0)
    images_list.append(image)
    labels_list.append(labels[i])
    
    print(i, '/', len(images))


transpose_images = np.array(images_list)
transpose_labels = np.array(labels_list)

# うまくいっていたら保存
print("transpose_images dtype: {}".format(type(transpose_images)))
print("transpose_images shape: {}".format(transpose_images.shape))
print("transpose_images: {}".format(transpose_images))

print("transpose_labels dtype: {}".format(type(transpose_labels)))
print("transpose_labels shape: {}".format(transpose_labels.shape))
print("transpose_labels: {}".format(transpose_labels))
np.savez(APP_PATH, 'data/temp/np_savez', transpose_images, datasets_npz_x1['y'])

# image = Image.fromarray(image)
# print("image dtype: {}".format(type(image)))
# image = image.convert("RGB")
# print("image mode: {}".format(image.mode)) # むりやり3チャンネルにしているので表記上はRGBだが実際はL
# image = np.asarray(image, np.uint8)
# print("image shape: {}".format(image.shape))
# image = np.transpose(image, (2, 0, 1))
# print("image transpose shape: {}".format(image.shape))

image-type： <class 'numpy.ndarray'>
image dtype: <class 'PIL.Image.Image'>
image mode: RGB
image shape: (960, 960, 3)
image transpose shape: (3, 960, 960)
0 / 290
image-type： <class 'numpy.ndarray'>
image dtype: <class 'PIL.Image.Image'>
image mode: RGB
image shape: (960, 960, 3)
image transpose shape: (3, 960, 960)
1 / 290
image-type： <class 'numpy.ndarray'>
image dtype: <class 'PIL.Image.Image'>
image mode: RGB
image shape: (960, 960, 3)
image transpose shape: (3, 960, 960)
2 / 290
image-type： <class 'numpy.ndarray'>
image dtype: <class 'PIL.Image.Image'>
image mode: RGB
image shape: (960, 960, 3)
image transpose shape: (3, 960, 960)
3 / 290
image-type： <class 'numpy.ndarray'>
image dtype: <class 'PIL.Image.Image'>
image mode: RGB
image shape: (960, 960, 3)
image transpose shape: (3, 960, 960)
4 / 290
image-type： <class 'numpy.ndarray'>
image dtype: <class 'PIL.Image.Image'>
image mode: RGB
image shape: (960, 960, 3)
image transpose shape: (3, 960, 960)
5 / 290
image-type： <class 'nu

In [ ]:
# transformsなしの自作のデータセット作成

data = torch.tensor(transpose_images, dtype=torch.float32)
label = torch.tensor(transpose_labels, dtype=torch.int64)

print('data.shape:', data.shape, 'label.shape:', label.shape)

# Datasetを作成
dataset = torch.utils.data.TensorDataset(data, label)

X_sample, y_sample = dataset[0]
print(X_sample.shape, y_sample.shape)
print(X_sample)
print(y_sample)



# transformsを使ったrailにのったやり方(自作クラスを後に作成する)
# https://dreamer-uma.com/pytorch-dataset/

data.shape: torch.Size([10, 3, 960, 960]) label.shape: torch.Size([10])
torch.Size([3, 960, 960]) torch.Size([])
tensor([[[ 65.,  54.,  47.,  ..., 117., 119., 115.],
         [ 74.,  62.,  53.,  ..., 110., 115., 107.],
         [ 74.,  88., 112.,  ..., 115., 113., 111.],
         ...,
         [ 80.,  79.,  89.,  ...,  90.,  98., 134.],
         [ 84.,  82., 100.,  ..., 239., 240., 235.],
         [ 85.,  84., 103.,  ..., 238., 236., 236.]],

        [[ 65.,  54.,  47.,  ..., 117., 119., 115.],
         [ 74.,  62.,  53.,  ..., 110., 115., 107.],
         [ 74.,  88., 112.,  ..., 115., 113., 111.],
         ...,
         [ 80.,  79.,  89.,  ...,  90.,  98., 134.],
         [ 84.,  82., 100.,  ..., 239., 240., 235.],
         [ 85.,  84., 103.,  ..., 238., 236., 236.]],

        [[ 65.,  54.,  47.,  ..., 117., 119., 115.],
         [ 74.,  62.,  53.,  ..., 110., 115., 107.],
         [ 74.,  88., 112.,  ..., 115., 113., 111.],
         ...,
         [ 80.,  79.,  89.,  ...,  90.,  98., 

In [ ]:
# DataLoader ミニバッチ学習
# 入力値と目標値をまとめる


# 学習データ、検証データ、テストデータに分ける
# 各データセットのサンプル数を決定
# train : val : test = 60% : 20% : 20%
num_train = int(len(dataset) * 0.8)
# num_val = int(len(dataset) * 0.2)
num_test = int(len(dataset) - num_train)

print('num_train:', num_train)
# print('num_val:', num_val)
print('num_test:', num_test)

# ランダムにするらめにシードの固定をした
torch.manual_seed(0)
# データセットの分割
train, test = torch.utils.data.random_split(dataset, [num_train, num_test])
print('train:', train)
# print('val:', val)
print('test:', test)




# 一括でデータを作成
# train_X, test_X, train_y, test_y = train_test_split(X, y, test_size
# =0.8, random_state=1)


num_train: 8
num_test: 2
train: <torch.utils.data.dataset.Subset object at 0x7fbf565dd9c0>
test: <torch.utils.data.dataset.Subset object at 0x7fbf565dc460>


In [ ]:
# ミニバッチ学習

# バッチサイズの定義(目安は全データの1/10だが正解がない)
batch_size = 2
train_loader = torch.utils.data.DataLoader(train, batch_size, shuffle=True, drop_last=True)
# val_loader = torch.utils.data.DataLoader(val, batch_size)
test_loader = torch.utils.data.DataLoader(test, batch_size)

# 1つ目のバッチサイズに分割したデータを確認
x, t = next(iter(train_loader))

print('x:', x)
print('t:', t)




x: tensor([[[[169., 169., 169.,  ..., 178., 169., 181.],
          [171., 171., 171.,  ..., 177., 171., 180.],
          [174., 174., 174.,  ..., 174., 175., 177.],
          ...,
          [ 95.,  87.,  80.,  ..., 175., 174., 174.],
          [109.,  89.,  88.,  ..., 173., 172., 172.],
          [151., 104.,  85.,  ..., 171., 170., 170.]],

         [[169., 169., 169.,  ..., 178., 169., 181.],
          [171., 171., 171.,  ..., 177., 171., 180.],
          [174., 174., 174.,  ..., 174., 175., 177.],
          ...,
          [ 95.,  87.,  80.,  ..., 175., 174., 174.],
          [109.,  89.,  88.,  ..., 173., 172., 172.],
          [151., 104.,  85.,  ..., 171., 170., 170.]],

         [[169., 169., 169.,  ..., 178., 169., 181.],
          [171., 171., 171.,  ..., 177., 171., 180.],
          [174., 174., 174.,  ..., 174., 175., 177.],
          ...,
          [ 95.,  87.,  80.,  ..., 175., 174., 174.],
          [109.,  89.,  88.,  ..., 173., 172., 172.],
          [151., 104.,  85.,  

In [ ]:
# 2.3 ニューラルネットワークの定義

num_classes = 6    # CIFAR10のクラスの数を指定

class AlexNet(nn.Module):
    def __init__(self, num_classes=num_classes):
        super(AlexNet, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=7, padding=2),     # 畳み込み層
            nn.ReLU(inplace=True),                          # 活性化関数
            nn.MaxPool2d(kernel_size=2),                    # プーリング層
            nn.Conv2d(64, 192, kernel_size=5, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2),
            nn.Conv2d(192, 384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(384, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2),
        )
        self.classifier = nn.Sequential(
            nn.Dropout(),                   # ドロップアウト層
            nn.Linear(256 * 3* 3, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096, num_classes),
        )

    # 順伝搬を定義
    def forward(self, x):
        x = self.features(x)                # 画像特徴量抽出パート
        x = x.view(x.size(0), 256 * 3 * 3)  # 3次元から1次元に変えて全結合層へ
        x = self.classifier(x)              # 画像分類パート
        return x
        
# ネットワークのロード
# CPUとGPUのどちらを使うかを指定
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
net = AlexNet().to(device)
print(net)
# デバイスの確認
print("Device: {}".format(device))


AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(1, 1), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=2304, out_features=4096, bias=True)
    (2): ReLU(inplace=True)
    (3): Dropout(p=0.5, in

In [ ]:
# 2.4 損失関数と最適化関数の定義

# 損失関数の定義
criterion = nn.CrossEntropyLoss()

# 活性化関数の定義
optimizer = optim.Adam(net.parameters())

In [ ]:
# 2.5 学習

# 損失と正解率を保持するリストを作成
train_loss_list = []        # 学習損失
train_accuracy_list = []    # 学習データの正答率
test_loss_list = []         # 評価損失
test_accuracy_list = []     # テストデータの正答率

# 学習(エポック)の実行
epoch = 10
for i in range(epoch):
    # エポックの進行状況を表示
    print("-------------------------------")
    print("Epoch: {}/{}".format(i+1, epoch))
    
    # 損失と正解率の初期化
    train_loss = 0      # 学習損失
    train_accuracy = 0  # 学習データの正答率
    test_loss = 0       # 評価損失
    test_accuracy = 0   # テストデータの正答率
    
    # ---------学習パート--------- #
    # ニューラルネットワークを学習モードに設定
    net.train()
    # ミニバッチごとにデータをロードし学習
    for images, labels in train_loader:
        # GPUにTensorを転送
        images = images.to(device)
        labels = labels.to(device)
        
        # 勾配を初期化
        optimizer.zero_grad()
        # データを入力して予測値を計算(順伝播)
        y_pred_prob = net(images)
        # 損失(誤差)を計算
        loss = criterion(y_pred_prob, labels)
        # 勾配の計算(逆伝搬)
        loss.backward()
        # パラメータ(重み)の更新
        optimizer.step()
        
        # ミニバッチごとの損失を蓄積
        train_loss += loss.item()
        
        # 予測したラベルを予測確率y_pred_probから計算
        y_pred_labels = torch.max(y_pred_prob, 1)[1]
        # ミニバッチごとに正解したラベル数をカウント
        train_accuracy += torch.sum(y_pred_labels == labels).item() / len(labels)
    
    # エポックごとの損失と正解率を計算(ミニバッチの平均の損失と正解率を計算)
    epoch_train_loss = train_loss / len(train_loader)
    epoch_train_accuracy = train_accuracy / len(train_loader)
    # ---------学習パートはここまで--------- #
    
    # ---------評価パート--------- #
    # ニューラルネットワークを評価モードに設定
    net.eval()
    # 評価時の計算で自動微分機能をオフにする
    with torch.no_grad():
        for images, labels in test_loader:
            # GPUにTensorを転送
            images = images.to(device)
            labels = labels.to(device)
            # データを入力して予測値を計算(順伝播)
            y_pred_prob = net(images)
            # 損失(誤差)を計算
            loss = criterion(y_pred_prob, labels)
            # ミニバッチごとの損失を蓄積
            test_loss += loss.item()
            
            # 予測したラベルを予測確率y_pred_probから計算
            y_pred_labels = torch.max(y_pred_prob, 1)[1]
            # ミニバッチごとに正解したラベル数をカウント
            test_accuracy += torch.sum(y_pred_labels == labels).item() / len(labels)
        
    # エポックごとの損失と正解率を計算(ミニバッチの平均の損失と正解率を計算)
    epoch_test_loss = test_loss / len(test_loader)
    epoch_test_accuracy = test_accuracy / len(test_loader)
    # ---------評価パートはここまで--------- #

    # エポックごとに損失と正解率を表示
    print("Train_Loss: {:.4f}, Train_Accuracy: {:.4f}".format(epoch_train_loss, epoch_train_accuracy))
    print("Test_Loss: {:.4f}, Test_Accuracy: {:.4f}".format(epoch_test_loss, epoch_test_accuracy))

    # 損失と正解率をリスト化して保存
    train_loss_list.append(epoch_train_loss)
    train_accuracy_list.append(epoch_train_accuracy)
    test_loss_list.append(epoch_test_loss)
    test_accuracy_list.append(epoch_test_accuracy)

-------------------------------
Epoch: 1/10


RuntimeError: ignored

In [ ]:
# 2.6 結果の可視化

# 損失
plt.figure()
plt.title('Train and Test Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.plot(range(1, epoch+1), train_loss_list, color='blue', linestyle='-', label='Train_Loss')
plt.plot(range(1, epoch+1), test_loss_list, color='red', linestyle='--', label='Test_Loss')
plt.legend()    # 凡例

# 正解率
plt.figure()
plt.title('Train and Test Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.plot(range(1, epoch+1), train_accuracy_list, color='blue', linestyle='-', label='Train_Accuracy')
plt.plot(range(1, epoch+1), test_accuracy_list, color='red', linestyle='--', label='Test_Accuracy')
plt.legend()    # 凡例

# 表示
plt.show()

In [ ]:
# 分類した画像を確認
# CIFAR10のクラス
# classes = ['plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']  # CIFAR10のクラス
CATEGORIES=["S10C","S15C","S25C","S35C","S45C","S55C"]


# ニューラルネットワークを評価モードに設定
net.eval()
# 評価時の計算で自動微分機能をオフにする
with torch.no_grad():
    for images, labels in test_loader:
        # GPUにTensorを転送
        images = images.to(device)
        labels = labels.to(device)
        # データを入力して予測値を計算(順伝播)
        y_pred_prob = net(images)
        # 予測したラベルを予測確率y_pred_probから計算
        y_pred_labels = torch.max(y_pred_prob, 1)[1]
        
        for i in range(9):
            image = images[i] / 2 + 0.5
            image = image.to('cpu').numpy()
            plt.subplot(3, 3, i+1)
            # matplotlibでは(縦, 横, チャネル)の順
            plt.imshow(np.transpose(image, (1, 2, 0)))
            plt.title(CATEGORIES[labels[i]])
            plt.title("{} ({})".format(CATEGORIES[y_pred_labels[i].item()], CATEGORIES[labels[i].item()]), color=('green' if y_pred_labels[i] == labels[i] else 'red'))
            plt.axis('off')
        plt.show()
        break
